### importamos las librerias a utilizar

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import os

### Preparacion de espacio de trabajo

especificamos la ruta de nuestro dataset, un arreglo para guardar las categorias que vayamos encontrando y la dimension a trabajar con nuestras imagenes

In [3]:
DATADIR = '/home/lenin/Documents/datasets/colores'
CATEGORIES = []
IMG_SIZE=30

recorremos el direcotorio excluyendo las imagenes de testeo

In [4]:
for cate in os.listdir(DATADIR):
    if cate == 'testimg': continue
    CATEGORIES.append(cate)
print(f'categorias encontradas: {CATEGORIES}\n total: {len(CATEGORIES)}')

categorias encontradas: ['morado', 'azul', 'rojo', 'amarillo', 'gris', 'rosado', 'negro', 'naranja', 'celeste', 'verde', 'blanco', 'cafe']
 total: 12


### Recoleccion de datos
recorremos el directorio tomando cada imagen segun su respectiva categoria

In [5]:
training_data=[] #var para los datos recolectados
labels=0
for category in CATEGORIES:
    path=os.path.join(DATADIR, category)
    for fname in os.listdir(path):
        img = load_img((path+'/'+fname), target_size=(IMG_SIZE,IMG_SIZE))
        x = img_to_array(img)
        x=x/255
        training_data.append([x,labels])
    labels+=1
print('done')

/home/lenin/.local/lib/python3.10/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


done


verificamos el total de datos que tenemos

In [6]:
lenofimage = len(training_data)
print(lenofimage)

2400


### Tratamiento de los datos
separamos  nuestra data en img-labels y los tratamos con numpy

In [7]:
X=[]
y=[]
for img, label in training_data:
    X.append(img)
    y.append(label)
print('done')

done


verificamos la forma de nuestra data

In [8]:
X=np.array(X)
y=np.array(y)
print('labels',y.shape)
print('img',X.shape)

labels (2400,)
img (2400, 30, 30, 3)


dividimos la data para train y test

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
print(f'train: {len(X_train)}, test: {len(X_test)}')

train: 1800, test: 600


### Creacion del modelo
creamos la arquitectura del modelo, lo compilamos y finalmente ajustamos

In [10]:
# model = Sequential()
# model.add(Flatten(input_shape=(X_train[0].shape)))
# model.add(Dense(len(CATEGORIES), activation='softmax'))


In [11]:
model = Sequential([
    Conv2D(32, (2,2), activation='relu', padding='SAME', input_shape=X_train[0].shape),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(len(CATEGORIES), activation='softmax'),
])

2023-02-02 15:39:48.093285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-02 15:39:48.093786: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-02 15:39:48.093933: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-02 15:39:48.094046: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-02 15:39:48.094158: W tensorflow/c

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 10)                72010     
                                                                 
 dense_1 (Dense)             (None, 12)                132       
                                                                 
Total params: 72,558
Trainable params: 72,558
Non-trainable params: 0
____________________________________________________

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=19, batch_size=25)

Epoch 1/19
72/72 [==============================] - 2s 14ms/step - loss: 1.9292 - accuracy: 0.3411
Epoch 2/19
72/72 [==============================] - 1s 14ms/step - loss: 1.1249 - accuracy: 0.6472
Epoch 3/19
72/72 [==============================] - 1s 14ms/step - loss: 0.8273 - accuracy: 0.7083
Epoch 4/19
72/72 [==============================] - 1s 14ms/step - loss: 0.6736 - accuracy: 0.7811
Epoch 5/19
72/72 [==============================] - 1s 14ms/step - loss: 0.5808 - accuracy: 0.8056
Epoch 6/19
72/72 [==============================] - 1s 14ms/step - loss: 0.4496 - accuracy: 0.8861
Epoch 7/19
72/72 [==============================] - 1s 14ms/step - loss: 0.3563 - accuracy: 0.9067
Epoch 8/19
72/72 [==============================] - 1s 14ms/step - loss: 0.3172 - accuracy: 0.9172
Epoch 9/19
72/72 [==============================] - 1s 14ms/step - loss: 0.3078 - accuracy: 0.9078
Epoch 10/19
72/72 [==============================] - 1s 14ms/step - loss: 0.2832 - accuracy: 0.9111
Epoch 11/

verificamos la precision en testeo

In [14]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print('test loss',test_loss)
print('test accuracy',test_accuracy)

test loss 0.20668433606624603
test accuracy 0.9449999928474426


### Probamos el modelo entrenado

In [18]:
#ruta a nuestras imagenes de test en el directorio, no de la data preparada
path = DATADIR + '/testimg' + '/1.jpg' 

#tratamos la img con el mismo tamanio y la normalizamos
img = load_img(path, target_size=(IMG_SIZE,IMG_SIZE))
x = img_to_array(img)
x=x/255

#agrego un eje para que el modelo lo reciba
x = x[np.newaxis, ...]

#imprimo la img y la prediccion
plt.imshow(img)
resp = model.predict(x, verbose=0)
print(f'prediccion = {CATEGORIES[np.argmax(resp[0])]}')

FileNotFoundError: [Errno 2] No such file or directory: '/home/lenin/Documents/datasets/colores/testimg/1.jpg'

### Exportacion
con el modelo ya funcionando correctamente, lo exportamos para poder utilizarlo en otros proyectos

In [16]:
# model.save('mod_color_v1.h5')
# model.save_weights("mod_color_v1_weights.h5")

para cargarlos

In [17]:
# # Cargar la estructura del modelo
# modelo_cargado = tf.keras.models.load_model("mod_color_v1.h5")

# # Cargar los pesos del modelo
# modelo_cargado.load_weights("mod_color_v1_weights.h5")
convd 
drop out 
max poling 
padding 
